In [28]:
import pandas as pd
df = pd.read_csv('output.csv')
df.shape

(289962, 11)

In [29]:
df.describe()

,lap_num,position_during_lap,age,start_pos
count,289962.000000,289952.000000,289962.000000,289962.000000
mean,30.254002,9.919956,27.957484,10.477163
std,18.281548,5.569781,5.358625,6.091154
min,1.000000,1.000000,17.000000,0.000000
25%,15.000000,5.000000,24.000000,5.000000
50%,29.000000,10.000000,27.000000,10.000000
75%,45.000000,14.000000,32.000000,16.000000
max,87.000000,24.000000,44.000000,24.000000


In [30]:
df.values

array([['00:01:40.424', 1, 1.0, ..., 1, '2024-09-22', nan],
       ['00:01:38.48', 2, 1.0, ..., 1, '2024-09-22', nan],
       ['00:01:38.406', 3, 1.0, ..., 1, '2024-09-22', nan],
       ...,
       ['00:01:31.812', 56, 7.0, ..., 22, '2012-03-18', nan],
       ['00:01:32.353', 57, 7.0, ..., 22, '2012-03-18', nan],
       ['00:01:40.163', 58, 8.0, ..., 22, '2012-03-18', nan]],
      dtype=object)

In [26]:
print(df['session_status'].unique())

['Finished' '+1 Lap' 'Power Unit' 'Accident' 'Retired' 'Overheating'
 '+2 Laps' 'Disqualified' 'Hydraulics' 'Water pressure' '+7 Laps'
 'Collision' 'Engine' 'Gearbox' 'Brakes' 'Collision damage' 'Withdrew'
 'Undertray' 'Radiator' 'Illness' 'Rear wing' 'Technical' 'Steering'
 'Oil leak' 'Power loss' 'Mechanical' 'Electrical' 'Water leak' 'Spun off'
 'Differential' '+6 Laps' 'Fuel pump' 'Vibrations' 'Turbo' 'Suspension'
 'Front wing' 'Fuel leak' 'Water pump' 'Cooling system' 'Fuel pressure'
 'Damage' 'Puncture' '+3 Laps' 'Wheel nut' 'Transmission' 'Debris'
 'Electronics' '+5 Laps' 'Wheel' 'Exhaust' 'Out of fuel' 'Battery' 'Tyre'
 'Spark plugs' 'Oil pressure' 'Seat' 'Driveshaft' '+4 Laps' 'Excluded'
 'Brake duct' 'Throttle' 'ERS' '+8 Laps' 'Drivetrain' 'Clutch'
 'Alternator' 'Pneumatics' '+11 Laps']


In [32]:
#Add field laps no nth since last pit of race start
df['laps_since_last_pit_or_start'] = 0

last_pit_lap = 0
for idx,  row in df.iterrows():
    if row['lap_num'] == 1:
        last_pit_lap = 1
        df.at[idx, 'laps_since_last_pit_or_start'] = last_pit_lap
    elif pd.notna(row['pitstop_duration']):
        df.at[idx, 'laps_since_last_pit_or_start'] = last_pit_lap
        last_pit_lap = 1
    else:
        df.at[idx, 'laps_since_last_pit_or_start'] = last_pit_lap
        last_pit_lap += 1

df



,laptime,lap_num,position_during_lap,driver_ref,age,team_ref,circuit_ref,session_status,start_pos,race_date,pitstop_duration,laps_since_last_pit_or_start
0,00:01:40.424,1,1.0,norris,25,mclaren,marina_bay,Finished,1,2024-09-22,NaN,1
1,00:01:38.48,2,1.0,norris,25,mclaren,marina_bay,Finished,1,2024-09-22,NaN,1
2,00:01:38.406,3,1.0,norris,25,mclaren,marina_bay,Finished,1,2024-09-22,NaN,2
3,00:01:38.446,4,1.0,norris,25,mclaren,marina_bay,Finished,1,2024-09-22,NaN,3
4,00:01:37.938,5,1.0,norris,25,mclaren,marina_bay,Finished,1,2024-09-22,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...
289957,00:01:31.621,54,7.0,perez,22,sauber,albert_park,Finished,22,2012-03-18,NaN,30
289958,00:01:31.559,55,7.0,perez,22,sauber,albert_park,Finished,22,2012-03-18,NaN,31
289959,00:01:31.812,56,7.0,perez,22,sauber,albert_park,Finished,22,2012-03-18,NaN,32
289960,00:01:32.353,57,7.0,perez,22,sauber,albert_park,Finished,22,2012-03-18,NaN,33


In [19]:
df.describe()

,lap_num,position_during_lap,age,start_pos,laps_since_last_pit_or_start
count,289962.000000,289952.000000,289962.000000,289962.000000,289962.000000
mean,30.254002,9.919956,27.957484,10.477163,13.652872
std,18.281548,5.569781,5.358625,6.091154,10.371022
min,1.000000,1.000000,17.000000,0.000000,1.000000
25%,15.000000,5.000000,24.000000,5.000000,6.000000
50%,29.000000,10.000000,27.000000,10.000000,11.000000
75%,45.000000,14.000000,32.000000,16.000000,19.000000
max,87.000000,24.000000,44.000000,24.000000,95.000000
